<a href="https://colab.research.google.com/github/beinghorizontal/wav2vec2/blob/main/create_n_grams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **3. Build an *n-gram* with KenLM**



Great, let's see step-by-step how to build an *n-gram*. We will use the popular [KenLM library](https://github.com/kpu/kenlm) to do so. Let's start by installing the Ubuntu library prerequisites:

In [2]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3ubuntu0.1).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.1).
zlib1g-dev set to manually in

before downloading and unpacking the KenLM repo.

In [3]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-08-17 11:47:59--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  --.-KB/s    in 0.06s   

2022-08-17 11:47:59 (8.05 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [4]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Boost: /usr/include (found suitable version "1.65.1", minimum required is "1.41.0") found components: program_options system

Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

In [15]:

!kenlm/build/bin/lmplz -o 5 <"/content/drive/MyDrive/textfile_ngram.txt" > "5gram.arpa" --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/textfile_ngram.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1918066688 bytes == 0x55df08748000 @  0x7fc44d1fb1e7 0x55df05fb47e2 0x55df05f4f4fe 0x55df05f2e2eb 0x55df05f1a066 0x7fc44b394c87 0x55df05f1bbaa
tcmalloc: large alloc 8950972416 bytes == 0x55df7ac7e000 @  0x7fc44d1fb1e7 0x55df05fb47e2 0x55df05fa380a 0x55df05fa4248 0x55df05f2e308 0x55df05f1a066 0x7fc44b394c87 0x55df05f1bbaa
****************************************************************************************************
Unigram tokens 193298 types 23799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:285588 2:1062820544 3:1992788608 4:3188461568 5:4649840128
tcmalloc: large alloc 4649844736 bytes == 0x55df08748000 @  0x7fc44d1fb1e7 0x55df05fb47e2 0x55df05fa380a 0x55df05fa4248 0x55df05f2e8d7 0x55df05f1a066 0x7fc44b394c87 0x55df05f1bbaa
tcmalloc: large alloc 

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [16]:
!head -20 5gram.arpa

\data\
ngram 1=23798
ngram 2=122559
ngram 3=174119
ngram 4=186458
ngram 5=188996

\1-grams:
-5.0976915	<unk>	0
0	<s>	-0.07110812
-4.710767	Hazelwood	-0.07110808
-4.4463267	swings	-0.07110812
-1.9540453	the	-0.30955175
-4.1648164	ball,	-0.093069255
-3.3074126	almost	-0.17228118
-4.4463267	beats	-0.116200306
-4.4463267	batsmen,	-0.07110812
-3.3947458	next	-0.17444298
-2.9169538	2	-0.16681048
-3.6837597	balls	-0.12040687


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [17]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [18]:
!head -20 5gram_correct.arpa

\data\
ngram 1=23799
ngram 2=122559
ngram 3=174119
ngram 4=186458
ngram 5=188996

\1-grams:
-5.0976915	<unk>	0
0	<s>	-0.07110812
0	</s>	-0.07110812
-4.710767	Hazelwood	-0.07110808
-4.4463267	swings	-0.07110812
-1.9540453	the	-0.30955175
-4.1648164	ball,	-0.093069255
-3.3074126	almost	-0.17228118
-4.4463267	beats	-0.116200306
-4.4463267	batsmen,	-0.07110812
-3.3947458	next	-0.17444298
-2.9169538	2	-0.16681048


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

### compress to binary

In [19]:
!kenlm/build/bin/build_binary /content/5gram_correct.arpa /content/5gram.bin

Reading /content/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
